데이터 로딩 관련 
https://leehyejin91.github.io/post-wide_n_deep/

In [75]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using {device} ")

using cuda 


In [110]:
df = pd.read_csv('/home/siyun/ephemeral/WDN/data/gift_data.csv')

In [111]:
def cat_stats(df) : 
    for col1 in df['category_1'].unique():
        # 특정 category_1 값에 해당하는 행들만 선택
        filtered_df1 = df[df['category_1'] == col1]

        # category_1에 대한 price 통계 계산
        mean_price1 = filtered_df1['price'].mean()
        std_price1 = filtered_df1['price'].std()

        mean_review1 = filtered_df1['num_review'].mean()
        std_review1 = filtered_df1['num_review'].std()

        # 결과 출력
        print(f"len : {len(filtered_df1)} || Category_1: {col1} || price_mean :{int(mean_price1)}({std_price1:.3f}) || price_min : {filtered_df1['price'].min()} || Q_0.3: {int(filtered_df1['price'].quantile(0.3))} || Q_0.5: {int(filtered_df1['price'].median())} || Q_0.99: {int(filtered_df1['price'].quantile(0.99))} || price_max : {filtered_df1['price'].max()}")

        print(f"                                        review_mean: {int(mean_review1)}({std_review1:.3f}) || review_min : {filtered_df1['price'].min()} || Q_0.3: {int(np.quantile(std_review1, 0.3))} || Q_0.5: {int(np.median(std_review1))}")
        # print(f"Q 0.3보다 낮은 row의 개수 : {len(filtered_df1[filtered_df1['price'] < filtered_df1['price'].quantile(0.3)])} 의 비율 : {len(filtered_df1[filtered_df1['price'] < filtered_df1['price'].quantile(0.3)]) / len(df)}")
        # category_2의 unique 값에 대해 반복
        for col2 in filtered_df1['category_2'].unique():
            # 특정 category_2 값에 해당하는 행들만 선택
            filtered_df2 = filtered_df1[filtered_df1['category_2'] == col2]

            if len(filtered_df2) <= 1 :
                pass
            else :
                # category_2에 대한 price 통계 계산
                mean_price2 = filtered_df2['price'].mean()
                std_price2 = filtered_df2['price'].std()


                mean_review2 = filtered_df2['num_review'].mean()
                std_review2 = filtered_df2['num_review'].std()
                
                # 각 카테고리1의 하위 카테고리인 카테고리2의 quantile 0.3 이하를 제거 

                # 결과 출력
                print(f"    len : {len(filtered_df2)} || Category_2: {col2} || Category_1: {col1} || price_mean :{int(mean_price2)}({std_price2:.3f}) || price_min : {filtered_df2['price'].min()} || Q_0.3: {int(filtered_df2['price'].quantile(0.3))} || Q_0.5: {int(filtered_df2['price'].median())} || Q_0.99: {int(filtered_df2['price'].quantile(0.99))} || price_max : {filtered_df2['price'].max()}")
                print(f"                                             review_mean: {int(mean_review2)}({std_review2:.3f}) || review_min : {filtered_df2['price'].min()} || Q_0.3: {int(np.quantile(std_review2, 0.3))} || Q_0.5: {int(np.median(std_review2))}")


개성 계산
- 엔트로피 계수를 이용해서 선택의 다양성을 계산합니다
- 높은 엔트로피 ~= 다양하게 선택됨
- 낮은 엔트로피 ~= 일관되게 선택됨 

In [112]:
# 각 product_id 별로 size(count) 계산

df['category_3'].fillna('unknown', inplace=True)
product_category_counts = df.groupby(['product_id', 'category_1', 'category_2', 'category_3']).size().reset_index(name='counts')

# 각 product_id 내에서 카테고리가 선택될 확률 계산
product_category_counts['total_counts'] = product_category_counts.groupby('product_id')['counts'].transform('sum')
product_category_counts['prob'] = product_category_counts['counts'] / product_category_counts['total_counts']

# 각 product_id에 대한 엔트로피를 계산
# -sigma(pi) * log2(pi)
product_category_counts['personality'] = -product_category_counts['prob'] * np.log2(product_category_counts['prob'])
product_entropy = product_category_counts.groupby('product_id')['personality'].sum().reset_index()

# 매핑
mean_entropy = product_entropy['personality'].mean()
df = df.merge(product_entropy, on='product_id', how='left')
df['personality'].fillna(mean_entropy, inplace=True)


In [113]:
len(df)

330180

In [114]:
df['product_id'].nunique()

131075

# Filtering

In [115]:
filtered_df = pd.DataFrame()

for col1 in df['category_1'].unique():
    # 특정 category_1 값에 해당하는 행들만 선택
    filtered_df1 = df[df['category_1'] == col1]

    # category_2의 unique 값에 대해 반복
    for col2 in filtered_df1['category_2'].unique():
        # 특정 category_2 값에 해당하는 행들만 선택
        filtered_df2 = filtered_df1[filtered_df1['category_2'] == col2]

        # 각 카테고리1의 하위 카테고리인 카테고리2의 quantile 0.3 이하를 제거 
        price_q_3 = filtered_df2['price'].quantile(0.3)
        price_q_99 =  filtered_df2['price'].quantile(0.99)
        rev_q_0_50 = filtered_df2['num_review'].quantile(0.5)

        filtered_q3 = filtered_df2[(filtered_df2['price'] > price_q_3) & (filtered_df2['price'] <= price_q_99) & (filtered_df2['num_review'] >= rev_q_0_50)]
        filtered_df = pd.concat([filtered_df, filtered_q3])


In [116]:
print(filtered_df.shape)
filtered_df['product_id'].nunique()


(115086, 12)


46094

In [117]:
# 카테고리1 별 비율 계산
cat1_ratios = filtered_df['category_1'].value_counts(normalize=True)

# 샘플링할 최종 데이터프레임 초기화
final_sample_df = pd.DataFrame()

# 총 샘플링할 개수
## 이상치를 제거한 나머지
total_samples = 30000

# 카테고리1 별로 루프
for cat1, cat1_ratio in cat1_ratios.items():
    cat1_df = filtered_df[filtered_df['category_1'] == cat1]
    # print(cat1_ratio)
    
    # 카테고리1의 비율에 따라 할당할 샘플 수 계산 (1 - 카테고리1의 비율)
    cat1_sample_size = int((cat1_ratio) * total_samples)
    # print(cat1_sample_size)
    
    # 카테고리2 별 비율 계산
    cat2_ratios = cat1_df['category_2'].value_counts(normalize=True)
    
    # 카테고리2 별로 샘플링
    for cat2, cat2_ratio in cat2_ratios.items():
        cat2_df = cat1_df[cat1_df['category_2'] == cat2]
        
        # 카테고리2 별 할당할 샘플 수 (카테고리2의 비율 * 카테고리1 내 샘플 수)
        cat2_sample_size = max(3, int(cat2_ratio * cat1_sample_size)) # 적어도 3개는 샘플링
        
        # 샘플링 후 최종 데이터프레임에 추가
        sampled_df = cat2_df.sample(n=cat2_sample_size, replace=False) # replace=True로 설정해서 데이터의 공백이 없도록 설정
        final_sample_df = pd.concat([final_sample_df, sampled_df])

# 최종 샘플링된 데이터프레임의 길이 조정
# 필요에 따라서는 여기서 추가 조정을 할 수 있습니다. 예를 들어 total_samples보다 많이 샘플링된 경우 조정 필요
if len(final_sample_df) > total_samples:
    final_sample_df = final_sample_df.sample(n=total_samples)


In [118]:
cat_stats(final_sample_df)

len : 11106 || Category_1: 생일선물 || price_mean :56194(92879.677) || price_min : 1980 || Q_0.3: 23900 || Q_0.5: 31500 || Q_0.99: 499000 || price_max : 1799000
                                        review_mean: 1930(2418.132) || review_min : 1980 || Q_0.3: 2418 || Q_0.5: 2418
    len : 2577 || Category_2: 환갑/칠순선물 || Category_1: 생일선물 || price_mean :81565(114899.983) || price_min : 19860 || Q_0.3: 29900 || Q_0.5: 39900 || Q_0.99: 659000 || price_max : 898000
                                             review_mean: 1976(2523.830) || review_min : 19860 || Q_0.3: 2523 || Q_0.5: 2523
    len : 2427 || Category_2: 4050대 여성 || Category_1: 생일선물 || price_mean :41610(32317.732) || price_min : 14990 || Q_0.3: 23500 || Q_0.5: 29900 || Q_0.99: 175217 || price_max : 230000
                                             review_mean: 2022(2523.108) || review_min : 14990 || Q_0.3: 2523 || Q_0.5: 2523
    len : 1681 || Category_2: 2030대 여성 || Category_1: 생일선물 || price_mean :42496(67143.434) || price_min : 

In [120]:
product_id_counts = final_sample_df['product_id'].value_counts()

# 빈도가 3 이상인 product_id 선택
valid_product_ids = product_id_counts[product_id_counts >= 2].index

# 해당 product_id만 포함하는 행을 필터링
final_sample_df_freq_2= final_sample_df[final_sample_df['product_id'].isin(valid_product_ids)]


print(f"filter_size : {final_sample_df_freq_2.shape}")
print(f"filter_product_unique : {final_sample_df_freq_2['product_id'].nunique()}")
print(f"filter_brand_unique : {final_sample_df_freq_2['brand'].nunique()}")


filter_size : (15550, 12)
filter_product_unique : 6084
filter_brand_unique : 2148


In [77]:
# print(f"filter_size : {final_sample_df_freq_2.shape}")
# print(f"filter_product_unique : {final_sample_df_freq_2['product_id'].nunique()}")
# print(f"filter_brand_unique : {final_sample_df_freq_2['brand'].nunique()}")


filter_size : (92356, 12)
filter_product_unique : 23365
filter_brand_unique : 5003


최종 sampling 저장

In [ ]:
final_sample_df_freq_2.to_csv('/home/siyun/ephemeral/data/filtered_gift.csv', index=False)


In [121]:
filtered_data = final_sample_df_freq_2


In [122]:
filtered_data.describe()


,product_id,rating,num_review,price,personality
count,1.555000e+04,15550.000000,15550.000000,1.555000e+04,15550.000000
mean,4.817781e+09,4.794135,2124.620836,5.686508e+04,2.722065
std,2.298036e+09,0.086865,2517.321270,1.292330e+05,0.690299
min,1.000013e+08,4.500000,229.000000,2.200000e+03,0.000000
25%,3.756750e+09,4.800000,609.000000,2.350000e+04,2.584963
50%,5.098248e+09,4.800000,1076.500000,3.300000e+04,2.807355
75%,6.115790e+09,4.800000,2326.750000,5.200000e+04,3.321928
max,9.783159e+09,5.000000,9999.000000,2.598000e+06,3.584963


라벨 인코딩

In [123]:
category_cols = ['category_1', 'category_2', 'category_3', 'brand']
# category_cols = ['category_1', 'category_2', 'category_3']
label_encoders = {}
for col in category_cols:
    unique_values = filtered_data[col].nunique()
    print(f"{col} unique values: {unique_values}")
    le = LabelEncoder()
    filtered_data[col] = le.fit_transform(filtered_data[col])
    label_encoders[col] = le
print("Final DataFrame shape after Label Encoding and dropping columns:", filtered_data.shape)

category_1 unique values: 9
category_2 unique values: 39
category_3 unique values: 48
brand unique values: 2148
Final DataFrame shape after Label Encoding and dropping columns: (15550, 12)


/tmp/ipykernel_1040716/686339282.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[col] = le.fit_transform(filtered_data[col])
/tmp/ipykernel_1040716/686339282.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[col] = le.fit_transform(filtered_data[col])
/tmp/ipykernel_1040716/686339282.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [100]:
filtered_data.head()


,product_id,product_name,brand,category_1,category_2,category_3,rating,num_review,price,image_url,product_url,personality
105978,4542290071,대웅생명과학 프리바이오틱스 프로바이오틱스 모유 유산균 신바이오틱스 6박스 (6개월분),1968,3,38,3,4.8,487,71900,https://shop-phinf.pstatic.net/20190801_89/pur...,https://shopping.naver.com/gift/products/45422...,2.807355
216384,5626140337,"로에 라일락 스카이 EDT 50ml / 라일락 향수, 생화향",635,3,38,46,4.7,690,42000,https://shop-phinf.pstatic.net/20231201_69/170...,https://shopping.naver.com/gift/products/56261...,2.807355
288492,7367947637,종근당건강 아임비타 마그네슘 1박스 (3개월분),1652,3,38,3,4.8,633,19900,https://shop-phinf.pstatic.net/20230629_186/16...,https://shopping.naver.com/gift/products/73679...,2.807355
179066,5184997505,코엔자임큐텐 코큐텐 2개월 높은 혈압감소 도움 코엔자임 q10 항산화에좋은 coq10,1008,3,38,3,4.8,7260,23800,https://shop-phinf.pstatic.net/20220404_116/16...,https://shopping.naver.com/gift/products/51849...,2.807355
290328,7453565810,미니막스 정글 멀티비타민 미네랄 복숭아맛 1통 60구미,516,3,38,3,4.9,3299,27600,https://shop-phinf.pstatic.net/20240124_134/17...,https://shopping.naver.com/gift/products/74535...,3.000000


In [124]:
filtered_data = filtered_data.drop(['image_url','product_url'], axis=1)
filtered_data


,product_id,product_name,brand,category_1,category_2,category_3,rating,num_review,price,personality
268848,6663913427,푸릇담 젊은농부 NFC 사과즙 30포+30포,1621,3,38,3,4.8,1229,32900,2.807355
103693,4508891313,그룬플러스 탈모완화 샴푸 토닉 세트 / 지루성두피 탈모케어,269,3,38,46,4.6,427,79000,3.000000
160442,5015177050,진생가 키즈로운 어린이홍삼 15ml 30포,1706,3,38,3,4.9,2087,78000,3.000000
247472,6110039794,녹십자웰빙 어삼 뉴질랜드 녹용 진액 선물세트 1박스 24포,53,3,38,3,4.9,437,29900,2.807355
266380,6597771892,GNM 상어연골 콘드로이친 1400 맥스 비타민D 60정 x 2병,55,3,38,3,4.8,1360,33900,2.807355
...,...,...,...,...,...,...,...,...,...,...
79687,3587601813,반다레더 자동차 가죽 스마트 차 키케이스 키커버 키홀더,886,2,31,1,4.9,3272,47000,1.000000
221790,5699511329,(B급) 누키노블 차 스마트 키케이스 자동차 가죽 키링,478,2,31,1,4.9,1345,20000,1.000000
256783,6346568325,스마트 차키케이스 가죽 키홀더 키커버 기아 KN-03,1774,2,31,1,4.8,734,27800,1.000000
280848,7012400866,오즈공방 자동차 스마트 차 키케이스 프랑스 최고급 가죽 키커버 BZ-03,1440,2,31,1,4.9,485,32000,1.000000


In [125]:
print("Final DataFrame shape after Label Encoding and dropping columns:", filtered_data.shape)

for col in category_cols:
    unique_values = filtered_data[col].nunique()
    print(f'{col} unique values:', unique_values)

# 데이터프레임의 현재 칼럼 출력
print("Current DataFrame columns:", filtered_data.columns.tolist())

Final DataFrame shape after Label Encoding and dropping columns: (15550, 10)
category_1 unique values: 9
category_2 unique values: 39
category_3 unique values: 48
brand unique values: 2148
Current DataFrame columns: ['product_id', 'product_name', 'brand', 'category_1', 'category_2', 'category_3', 'rating', 'num_review', 'price', 'personality']


In [126]:
filtered_data['price']

268848    32900
103693    79000
160442    78000
247472    29900
266380    33900
          ...  
79687     47000
221790    20000
256783    27800
280848    32000
126781    32000
Name: price, Length: 15550, dtype: int64

num_review, price에 대한 standardscaling

In [13]:
# # https://leehah0908.tistory.com/2

# from sklearn.preprocessing import StandardScaler

# s1 = StandardScaler()
# s2 = StandardScaler()

# s1.fit(df['price'].values.reshape(-1, 1))
# s2.fit(df['num_review'].values.reshape(-1, 1))

# price = s1.transform(df['price'].values.reshape(-1, 1))
# num_review = s2.transform(df['num_review'].values.reshape(-1, 1))

# df['price_scaling'] = price
# df['review_scaling'] = num_review

In [104]:
print(filtered_data.dtypes)
print(filtered_data.describe())
print(filtered_data.shape)

product_id        int64
product_name     object
brand             int64
category_1        int64
category_2        int64
category_3        int64
rating          float64
num_review        int64
price             int64
personality     float64
dtype: object
         product_id         brand    category_1    category_2    category_3  \
count  1.558400e+04  15584.000000  15584.000000  15584.000000  15584.000000   
mean   4.800460e+09   1026.636614      3.489990     17.570585     20.026566   
std    2.296516e+09    618.767879      2.168397     12.831400     20.027852   
min    1.001230e+08      0.000000      0.000000      0.000000      0.000000   
25%    3.629840e+09    467.000000      3.000000      4.000000      1.000000   
50%    5.082555e+09   1022.000000      3.000000     17.000000     10.000000   
75%    6.148406e+09   1556.000000      5.000000     26.000000     46.000000   
max    9.783159e+09   2142.000000      8.000000     38.000000     47.000000   

             rating    num_review 

In [127]:
id2idx = {id: idx for idx, id in enumerate(sorted(filtered_data['product_id'].unique()))}
idx2id = {idx: id for id, idx in id2idx.items()}
filtered_data['product_idx'] = filtered_data['product_id'].map(id2idx)


In [128]:
filtered_data['product_idx'].sort_values()

4            0
3            0
141          1
147          1
229          2
          ... 
330009    6083
330012    6083
330015    6083
330006    6083
330013    6083
Name: product_idx, Length: 15550, dtype: int64

In [80]:
# # 특성 및 타겟 선택
# X = filtered_data[['category_1', 'category_2', 'category_3', 'rating', 'num_review', 'price', 'personality']]
# y = filtered_data['product_idx']

# # 데이터 분할
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=42)

# # 훈련 데이터와 검증 데이터의 차원 확인
# print("X_train shape:", X_train.shape, "| X_train dtype:", X_train.dtypes)
# print(f'y_train shape : {y_train.shape}')
# print("X_val shape: ", X_val.shape, "| X_val dtype:", X_val.dtypes)

# # 훈련 라벨과 검증 라벨의 차원 확인
# print("y_train shape:", y_train.shape, "| y_train dtype:", y_train.dtype)
# print("y_val shape:", y_val.shape, "| y_val dtype:", y_val.dtype)

# for col in X.columns:
#     unique_values = filtered_data[col].nunique()
#     print(f'{col} unique values:', unique_values, "| dtype:", filtered_data[col].dtype)

X_train shape: (66883, 7) | X_train dtype: category_1       int64
category_2       int64
category_3       int64
rating         float64
num_review       int64
price            int64
personality    float64
dtype: object
y_train shape : (66883,)
X_val shape:  (3521, 7) | X_val dtype: category_1       int64
category_2       int64
category_3       int64
rating         float64
num_review       int64
price            int64
personality    float64
dtype: object
y_train shape: (66883,) | y_train dtype: int64
y_val shape: (3521,) | y_val dtype: int64
category_1 unique values: 9 | dtype: int64
category_2 unique values: 36 | dtype: int64
category_3 unique values: 37 | dtype: int64
rating unique values: 6 | dtype: float64
num_review unique values: 5659 | dtype: int64
price unique values: 1720 | dtype: int64
personality unique values: 11 | dtype: float64


In [78]:
# categorical_features = ['brand', 'category_1', 'category_2', 'category_3']
# continuous_features = ['rating', 'num_review', 'price', 'personality']

# # StandardScaler for continuous features
# scaler = StandardScaler()
# X_train[continuous_features] = scaler.fit_transform(X_train[continuous_features])
# X_val[continuous_features] = scaler.transform(X_val[continuous_features])

한 세트(03.10)
- 일단 split하고 loader에 넣고 model까지는 성공했음.
- 이제 다음 단계 진행 중

In [53]:
filtered_data.head()

,product_id,product_name,brand,category_1,category_2,category_3,rating,num_review,price,personality,product_idx
36565,685890861,[1+1행사] 빨간 적양배추즙 100ml 60포 무농약 원물,1939,3,41,4,4.8,1697,31800,2.807355,2893
221024,5685939646,마몽드 레티놀 앰플 토너 150ml X 2개 더블세트 순수레티놀 수분탄력 산뜻촉촉,1644,3,41,51,4.8,809,23400,3.459432,16766
316929,8702897045,[N페이 1천점+포토리뷰 1천점 적립] 한율 부들밤 모공수축 패드 50매 (150ml),4798,3,41,51,4.8,540,21750,3.459432,22726
298908,7837034070,종근당 이뮨 듀오 멀티 비타맥스 5박스 물없이 마시는 액상 비타민 올인원 종합 영양...,1751,3,41,4,4.9,3282,112400,3.000000,21756
277984,6933289395,[수면건강기능식품] 바이탈뷰티 굿슬립 가바365 90일+20일 (L-글루탐산발효가바분말),2841,3,41,4,4.8,456,83250,2.807355,20484


In [16]:
# categorical_columns = ['brand','category_1', 'category_2', 'category_3']
# continuous_columns = ['rating', 'num_review', 'price', 'personality']

# # 데이터 분할과 스케일링
# X = filtered_data[categorical_columns + continuous_columns]
# y = filtered_data['product_idx']

# # 카테고리형 데이터와 연속형 데이터 분리
# X_categorical = X[categorical_columns]
# X_continuous = X[continuous_columns]

# # feature값 스케일링
# scaler = StandardScaler()
# X_continuous_scaled = scaler.fit_transform(X_continuous)
# X_continuous_scaled = pd.DataFrame(X_continuous_scaled, columns=continuous_columns)

# X_cat_train, X_cat_val, X_cont_train, X_cont_val, y_train, y_val = train_test_split(X_categorical, X_continuous_scaled, y, test_size=0.2, random_state=42)


NameError: name 'StandardScaler' is not defined

data split을 진행한 후 

카테고리 + 연속형 변수

In [107]:
filtered_data['product_idx'].sort_values()

143          0
144          0
164          1
162          1
163          1
          ... 
329676    6125
329702    6126
329695    6126
330016    6127
330013    6127
Name: product_idx, Length: 15584, dtype: int64

In [55]:
filtered_data.head()

,product_id,product_name,brand,category_1,category_2,category_3,rating,num_review,price,personality,product_idx
36565,685890861,[1+1행사] 빨간 적양배추즙 100ml 60포 무농약 원물,1939,3,41,4,4.8,1697,31800,2.807355,2893
221024,5685939646,마몽드 레티놀 앰플 토너 150ml X 2개 더블세트 순수레티놀 수분탄력 산뜻촉촉,1644,3,41,51,4.8,809,23400,3.459432,16766
316929,8702897045,[N페이 1천점+포토리뷰 1천점 적립] 한율 부들밤 모공수축 패드 50매 (150ml),4798,3,41,51,4.8,540,21750,3.459432,22726
298908,7837034070,종근당 이뮨 듀오 멀티 비타맥스 5박스 물없이 마시는 액상 비타민 올인원 종합 영양...,1751,3,41,4,4.9,3282,112400,3.000000,21756
277984,6933289395,[수면건강기능식품] 바이탈뷰티 굿슬립 가바365 90일+20일 (L-글루탐산발효가바분말),2841,3,41,4,4.8,456,83250,2.807355,20484


In [131]:
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader, Dataset
categorical_columns = ['brand','category_1', 'category_2', 'category_3']
# categorical_columns = ['category_1', 'category_2', 'category_3']
continuous_columns = ['rating', 'num_review', 'price', 'personality']

def dl_data_split(data, test_size=0.4, seed=42):
    X = data[categorical_columns + continuous_columns]
    y = data['product_idx']

    X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=test_size, random_state=seed, stratify=y
)

    # 연속형 데이터 스케일링
    scaler = StandardScaler()
    X_train[continuous_columns] = scaler.fit_transform(X_train[continuous_columns])
    X_valid[continuous_columns] = scaler.transform(X_valid[continuous_columns])
    
    return X_train, X_valid, y_train, y_valid

X_train, X_valid, y_train, y_valid = dl_data_split(filtered_data)

# 기본제공되는 TensorDataset은 categorical과 continuous를 나누어서 받지 못함. 따라서 customdataset이 필요
class Cat_Con_Dataset(Dataset):
    def __init__(self, X, y):
        self.X_cat = torch.tensor(X[categorical_columns].values, dtype=torch.long)
        self.X_cont = torch.tensor(X[continuous_columns].values, dtype=torch.float)
        self.y = torch.tensor(y.values, dtype=torch.long)
        print("Label sample in dataset initialization:", self.y[:10])

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X_cat[idx], self.X_cont[idx], self.y[idx]

print(f"y_train Min: {y_train.min()}, Max: {y_train.max()}")
print(f"y_valid Min: {y_valid.min()}, Max: {y_valid.max()}")

# 데이터 로더 생성
def dl_data_loader(X_train, y_train, X_valid, y_valid, batch_size=64, shuffle=True):
    train_dataset = Cat_Con_Dataset(X_train, y_train)
    valid_dataset = Cat_Con_Dataset(X_valid, y_valid)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    
    return train_dataloader, valid_dataloader

train_dataloader, valid_dataloader = dl_data_loader(X_train, y_train, X_valid, y_valid)


train_dataset = Cat_Con_Dataset(X_train, y_train)
valid_dataset = Cat_Con_Dataset(X_valid, y_valid)

y_train Min: 0, Max: 6083
y_valid Min: 0, Max: 6083
Label sample in dataset initialization: tensor([ 940, 5636, 3369, 5486, 4741, 5310, 2710, 1059, 3711,  997])
Label sample in dataset initialization: tensor([3390,  838, 5073,  808, 1127, 2711,  319, 1266, 3482,  956])
Label sample in dataset initialization: tensor([ 940, 5636, 3369, 5486, 4741, 5310, 2710, 1059, 3711,  997])
Label sample in dataset initialization: tensor([3390,  838, 5073,  808, 1127, 2711,  319, 1266, 3482,  956])


In [58]:
X_train
X_valid 
y_train
y_valid

98211      7774
166707    12833
203408    15554
157908    12233
90399      7140
          ...  
235725    17805
24077      1936
328229    23294
6413        477
185204    14235
Name: product_idx, Length: 24013, dtype: int64

In [132]:
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

print(f"y_train Min: {y_train.min()}, Max: {y_train.max()}")
print(f"y_valid Min: {y_valid.min()}, Max: {y_valid.max()}")

(9330, 8)
(6220, 8)
(9330,)
(6220,)
y_train Min: 0, Max: 6083
y_valid Min: 0, Max: 6083


위에서 이 부분을 변형함

In [86]:
# from torch.utils.data import TensorDataset, DataLoader, Dataset


# def dl_data_split(data, test_size=0.2, seed=42):
#     X_train, X_valid, y_train, y_valid = train_test_split(
#         data.drop(['product_idx','product_id','num_review','price'], axis=1),
#         data['product_idx'],
#         test_size=test_size,
#         random_state=seed,
#         shuffle=True
#     )
#     return X_train, X_valid, y_train, y_valid

# # 데이터 분할 실행
# X_train, X_valid, y_train, y_valid = dl_data_split(df)

# def dl_data_loader(X_train, y_train, X_valid, y_valid, batch_size=64, shuffle=True):
#     train_dataset = TensorDataset(torch.tensor(X_train.values, dtype=torch.float), torch.tensor(y_train.values, dtype=torch.long))
#     valid_dataset = TensorDataset(torch.tensor(X_valid.values, dtype=torch.float), torch.tensor(y_valid.values, dtype=torch.long))
    
#     train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
#     valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    
#     return train_dataloader, valid_dataloader

# # 데이터 로더 실행
# train_dataloader, valid_dataloader = dl_data_loader(X_train, y_train, X_valid, y_valid)


bookrating 참고 + wide deep api
- https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.10/tensorflow/g3doc/tutorials/wide_and_deep/index.md

In [133]:
import wandb
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [149]:
class WideAndDeepModel(nn.Module):
    def __init__(self, categorical_field_dims, continuous_field_dims, embed_dim, mlp_dims, dropout):
        super().__init__()
        self.categorical_embedding = nn.ModuleList([
            nn.Embedding(num_embeddings=dim, embedding_dim=embed_dim) 
            for dim in categorical_field_dims])
        self.continuous_linear = nn.Linear(len(continuous_field_dims), embed_dim)
        total_embed_dim = len(categorical_field_dims) * embed_dim + embed_dim # 모든 연속형 특성을 하나의 차원으로 임베딩
        self.mlp = nn.Sequential(
            nn.Linear(total_embed_dim, mlp_dims[0]),
            nn.ReLU(),
            nn.Dropout(dropout),
            *[nn.Sequential(nn.Linear(mlp_dims[i], mlp_dims[i+1]), nn.ReLU(), nn.Dropout(dropout)) 
              for i in range(len(mlp_dims)-1)],
            nn.Linear(mlp_dims[-1], 6084)
        )

    def forward(self, x_categorical, x_continuous):
        x_embedded = [embedding(x_categorical[:, i]) for i, embedding in enumerate(self.categorical_embedding)]
        x_embedded = torch.cat(x_embedded, dim=1)
        x_continuous = self.continuous_linear(x_continuous)
        x_total = torch.cat([x_embedded, x_continuous], dim=1)
        x_mlp = self.mlp(x_total)
        output = F.softmax(x_mlp, dim=1)
        return output

# 모델 초기화
# categorical_field_dims = [df['brand'].nunique(), df['category_1'].nunique(), df['category_2'].nunique(), df['category_3'].nunique(), df['product_idx'].nunique()]
# categorical_field_dims = [filtered_data['brand'].nunique(), filtered_data['category_1'].nunique(), filtered_data['category_2'].nunique(), filtered_data['category_3'].nunique()]
categorical_field_dims = [filtered_data['brand'].nunique(), filtered_data['category_1'].nunique(), filtered_data['category_2'].nunique(), filtered_data['category_3'].nunique()]
continuous_field_dims = ['rating', 'personality', 'price_scaling', 'review_scaling']
model = WideAndDeepModel(categorical_field_dims, 
                         continuous_field_dims, 
                         embed_dim=32, mlp_dims=[256,256,128,128,64,64,32,32,16,16],
                         dropout=0.1).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-7)
criterion = nn.CrossEntropyLoss()

In [ ]:
# class WideAndDeepModel(nn.Module):
#     def __init__(self, categorical_field_dims, continuous_field_dims, embed_dim, mlp_dims, dropout):
#         super().__init__()
#         self.categorical_embedding = nn.ModuleList([nn.Embedding(num_embeddings=dim, embedding_dim=embed_dim) for dim in categorical_field_dims])
#         self.continuous_linear = nn.Linear(len(continuous_field_dims), 1)
#         total_embed_dim = len(categorical_field_dims) * embed_dim + len(continuous_field_dims)
#         self.mlp = nn.Sequential(
#             nn.Linear(total_embed_dim, mlp_dims[0]),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             *[nn.Sequential(nn.Linear(mlp_dims[i], mlp_dims[i+1]), nn.ReLU(), nn.Dropout(dropout)) for i in range(len(mlp_dims)-1)],
#             nn.Linear(mlp_dims[-1], 15641)
#         )

#     def forward(self, x_categorical, x_continuous):
#         # print("Input x_categorical:", x_categorical.shape)
#         # print("Input x_continuous:", x_continuous.shape)
#         x_embedded = [embedding(x_categorical[:, i]) for i, embedding in enumerate(self.categorical_embedding)]
#         x_embedded = torch.cat(x_embedded, dim=1)
#         # print("After embedding and concatenation:", x_embedded.shape)
#         x_total = torch.cat([x_embedded, x_continuous], dim=1)
#         # print("After concatenating with continuous:", x_total.shape)
#         x_linear = self.continuous_linear(x_continuous)
#         # print("Output of continuous linear:", x_linear.shape)
#         x_mlp = self.mlp(x_total)
#         # print("Output of MLP:", x_mlp.shape)
#         output = x_linear + x_mlp
#         # print("Final output:", output.shape)
#         return output

# # 모델 초기화
# # categorical_field_dims = [df['brand'].nunique(), df['category_1'].nunique(), df['category_2'].nunique(), df['category_3'].nunique(), df['product_idx'].nunique()]
# # categorical_field_dims = [filtered_data['brand'].nunique(), filtered_data['category_1'].nunique(), filtered_data['category_2'].nunique(), filtered_data['category_3'].nunique()]
# categorical_field_dims = [filtered_data['brand'].nunique(), filtered_data['category_1'].nunique(), filtered_data['category_2'].nunique(), filtered_data['category_3'].nunique()]
# continuous_field_dims = ['rating', 'personality', 'price_scaling', 'review_scaling']
# model = WideAndDeepModel(categorical_field_dims, 
#                          continuous_field_dims, 
#                          embed_dim=512, mlp_dims=[516,258,128,64],
#                          dropout=0.1).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-7)
# criterion = nn.CrossEntropyLoss()

In [150]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

WideAndDeepModel(
  (categorical_embedding): ModuleList(
    (0): Embedding(2148, 32)
    (1): Embedding(9, 32)
    (2): Embedding(39, 32)
    (3): Embedding(48, 32)
  )
  (continuous_linear): Linear(in_features=4, out_features=32, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=160, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
    )
    (4): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
    )
    (5): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
    )
    (6): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
    )
    (7): Sequential(

최적의 파라미터를 찾은 후 이걸로 학습

In [26]:
# def accuracy(outputs, labels):
#     _, preds = torch.max(outputs, dim=1)
#     return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# epochs = 10

# # train
# for epoch in range(epochs):
#     model.train()  # train mode
#     total_loss = 0
#     total_acc = 0

#     for batch in train_dataloader:
#         optimizer.zero_grad()  

#         x_categorical, x_continuous, labels = batch
#         x_categorical, x_continuous, labels = x_categorical.to(device), x_continuous.to(device), labels.to(device)

#         # 모델 실행 및 손실 계산
#         outputs = model(x_categorical, x_continuous)
#         loss = criterion(outputs, labels)
#         acc = accuracy(outputs, labels)



#         loss.backward()  
#         optimizer.step()  # parameter update

#         total_loss += loss.item() # loss 추가
#         total_acc += acc.item()

#     avg_loss = total_loss / len(train_dataloader)
#     avg_acc = total_acc / len(train_dataloader)

    
#     print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss}, ACC: {avg_acc}')
    



Epoch 1/10, Loss: 9.43476905562956, ACC: 8.867763904653802e-05
Epoch 2/10, Loss: 9.328825553348468, ACC: 0.00019509080590238365
Epoch 3/10, Loss: 9.126447314977916, ACC: 0.0003015039727582293
Epoch 4/10, Loss: 8.851553355118474, ACC: 0.0004788592508513053
Epoch 5/10, Loss: 8.631382509744668, ACC: 0.0006030079455164586
Epoch 6/10, Loss: 8.483264911188305, ACC: 0.0004965947786606129
Epoch 7/10, Loss: 8.333885899196282, ACC: 0.0007980987514188422
Epoch 8/10, Loss: 8.176499805168993, ACC: 0.0009754540295119182
Epoch 9/10, Loss: 8.04369274414357, ACC: 0.0011882803632236095
Epoch 10/10, Loss: 7.906053021871543, ACC: 0.00125922247446084


sweep를 이용한 finetuning

In [57]:
import sys
print(sys.executable)
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/home/siyun/ephemeral/miniconda3/envs/recbole/bin/python


wandb: Currently logged in as: tldbs5026. Use `wandb login --relogin` to force relogin


True

top k개의 acc 반환 함수

In [151]:
def topkacc(output, target, topk=(1,)) :
    with torch.no_grad() :
        maxk = max(topk)
        batch_size = target.size(0)

        # https://pytorch.org/docs/stable/generated/torch.topk.html
        ## torch.topk(input, k, dim=None, largest=True, sorted=True, *, out=None)
        ## top k개를 가져온 후
        _, pred = output.topk(maxk, 1, True, True)
        # https://pytorch.org/docs/stable/generated/torch.t.html
        # Expects input to be <= 2-D tensor and transposes dimensions 0 and 1.
        # True or False로 반환
        pred = pred.t()
        # 그리고 그 갯수를 맞춤
        ## https://pytorch.org/docs/stable/generated/torch.eq.html#torch.eq
        ## Computes element-wise equality
        correct = pred.eq(target.view(1,-1).expand_as(pred))

        res = []
        for k in topk : 
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100/batch_size))
        return res

In [152]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(outputs, labels):
    _, preds = torch.topk(outputs, 10, dim=1)
    preds = preds.cpu().numpy()
    labels = labels.cpu().numpy()
    
    # 상위 10개의 예측을 고려하여 recall@5 계산
    recalls = []
    for i in range(len(preds)):
        true_positives = 0
        for pred in preds[i]:
            if pred in labels[i]:
                true_positives += 1
        recalls.append(true_positives / len(labels[i]))

    recall_at_10 = sum(recalls) / len(recalls)

    # 다른 메트릭 계산
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)

    return precision, recall, f1, recall_at_10


In [154]:
def train():
    # WandB 초기화
    wandb.init(project="wide deep model sweep", config={
        "batch_size": [256,512,1024],
        "lr": [0.005,0.001,0.0005,0.0001],
        "weight_decay": [1e-5,1e-3],
        "embed_dim": [256,128],
        "dropout": 0.1,
        "mlp_dims": [[256,128,64,32], [128,64,32,16]],
        "epochs": [100,200,300]
    })

    # Config 가져오기
    config = wandb.config

    # 모델 및 데이터 로드
    # 모델 초기화
    model = WideAndDeepModel(categorical_field_dims, 
                             continuous_field_dims, 
                             embed_dim=config.embed_dim, 
                             mlp_dims=config.mlp_dims, 
                             dropout=config.dropout).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

    # 옵티마이저 및 손실 함수 정의
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    criterion = nn.CrossEntropyLoss()

    # 학습 코드 구현
    total_batches = 0  # total_batches 변수 선언 및 초기화
    for epoch in range(config.epochs):
        model.train()
        total_loss = 0
        top1 = 0
        top10 = 0
        for batch_idx, batch in enumerate(train_dataloader):
            # 학습 코드 작성
            optimizer.zero_grad()  
            x_cat, x_con, labels = batch
            x_cat, x_con, labels = x_cat.to(device), x_con.to(device), labels.to(device)
            outputs = model(x_cat, x_con)
            loss = criterion(outputs, labels)
            loss.backward()  
            optimizer.step()  
            total_loss += loss.item()

            # 정확도 계산
            precision, recall, f1 = calculate_metrics(outputs, labels)
            acc1, acc10 = topkacc(outputs, labels, topk=(1,10))
            top1 += acc1.item()
            top10 += acc10.item()
            # _, predicted = torch.max(outputs, 1)
            # batch_acc = (predicted == labels).sum().item() / len(labels)
            # total_acc += batch_acc
            # total_batches += 1

            # WandB에 배치별 정확도 기록
            # wandb.log({f"batch_{batch_idx}_accuracy": batch_acc})
        
        avg_loss = total_loss / len(train_dataloader)
        avg_top1 = top1 / len(train_dataloader)
        avg_top10 = top10 /len(train_dataset)
        

        # wandb에 로그 기록
        wandb.log({"epoch": epoch, "loss": avg_loss, "accuracy1": avg_top1, "accuracy10" : avg_top10, "precision": precision, "recall": recall, "f1_score": f1})

# WandB sweep 실행
wandb.agent(wandb.sweep({"name": "Wide and Deep Model Sweep_epoches", "method": "grid", "parameters": {
    "batch_size": {"values": [256,512,1024]},
    "lr": {"values": [0.005,0.001,0.0005,0.0001]},
    "weight_decay": {"values": [1e-5,1e-3]},
    "embed_dim": {"values": [256,128]},
    "dropout": {"values": [0.1, 0.2, 0.3]},
    "mlp_dims": {"values":[[256,128,64,64,32,32], [256,256,128,128,64,64,32,32,16,16]]},
    "epochs": {"values": [100,200,300]}
}}), function=train)



Create sweep with ID: vcevyz8u
Sweep URL: https://wandb.ai/tldbs5026/uncategorized/sweeps/vcevyz8u


wandb: Agent Starting Run: pno6gkw4 with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.1
wandb: 	embed_dim: 256
wandb: 	epochs: 100
wandb: 	lr: 0.005
wandb: 	mlp_dims: [256, 128, 64, 64, 32, 32]
wandb: 	weight_decay: 1e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


accuracy1,▁▁▁▁▁▆▁▁▁▁▁▁▁▁▃▁▃▁▆▁▁█▁▁▁▁▁▁▁▃▁▆▁▆▁▃▁▁▆▁
accuracy10,█▃▄▆▃▄▆▂▃▃▃▂▃▃▄▂▄▃▅▃▃▄▃▃▁▂▃▃▆▂▅▄▃▃▂▅▃▃▃▄
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁███████████████████████████████████████
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy1,0.0
accuracy10,0.00067
epoch,99
f1_score,0.0


wandb: Agent Starting Run: p49l9i9p with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.1
wandb: 	embed_dim: 256
wandb: 	epochs: 100
wandb: 	lr: 0.005
wandb: 	mlp_dims: [256, 128, 64, 64, 32, 32]
wandb: 	weight_decay: 0.001
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


accuracy1,▅▁▁▅█▅▁█▁▅▁▁▁▁▁▁▁▁█▅▅▁█▅▁▅▅▁▁▁▁▁▁▅▁▅▁▅▅▁
accuracy10,█▆▄▆▅▄▃▄▁▅▂▄▆▁▁▁▄▁▆▃▂▄▄▃▅▂▅▄▂▁▃▂▂▇▄█▃▇▂▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁███████████████████████████████████████
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy1,0.0
accuracy10,0.0005
epoch,99
f1_score,0.0


...

wandb: Agent Starting Run: gp5zdnqs with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.1
wandb: 	embed_dim: 256
wandb: 	epochs: 300
wandb: 	lr: 0.0005
wandb: 	mlp_dims: [256, 256, 128, 128, 64, 64, 32, 32, 16, 16]
wandb: 	weight_decay: 1e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


accuracy1,▄▅▂▄▇▂▂▂▂▂▅▁▁▂▄▄▁▂▁▄▄▂▁▂█▂▁▄▂▂▅▂▁▁▅▂▂▁▄▂
accuracy10,▇█▅▅▄▄▄▁▃█▄▃▄▃▂▂▂▁▅█▄▄▄▃▄▃▂▆▃▃▄▃▃▅▆▁▃▄▄▃
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▅█▁▁████▅█▁▅▅▅█▅██▅▅▅█▅▅▅▅█▅▅▅█▅█▁██▅█▅
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy1,0.0107
accuracy10,0.00167
epoch,299
f1_score,0.0


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 13svh4p2 with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.1
wandb: 	embed_dim: 256
wandb: 	epochs: 300
wandb: 	lr: 0.0005
wandb: 	mlp_dims: [256, 256, 128, 128, 64, 64, 32, 32, 16, 16]
wandb: 	weight_decay: 0.001
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Ctrl + C detected. Stopping sweep.


accuracy1,▁▆█▆▆▁▃▁▁▆█▁▁▁▆▃▁▁▁▃▁▆▆▃▃▁▁▃▃▁▁▁▁▁▁▃▃▆▁▁
accuracy10,▅█▇▆▄▂▂▂▄▃▄▁▃▂▂▁▂▂▂▂▁▂▂▂▃▂▃▂▆▄▃▃▂▂▄▃▄▃▂▂
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁▁▁█████████████████████████████████████
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy1,0.0
accuracy10,0.00084
epoch,104
f1_score,0.0


모델의 output을 바탕으로 top k개를 추천

In [27]:
# # 모든 데이터에 대해서 가장 인기많은 top_10 반환하기
# from collections import Counter

# def predict_most_top_k_products(model, dataloader, k=10):
#     model.eval()
#     all_top_k_indices = []
    
#     with torch.no_grad():
#         for batch in dataloader:
#             x_categorical, x_continuous, _ = batch
#             x_categorical = x_categorical.to(device)
#             x_continuous = x_continuous.to(device)
            
#             outputs = model(x_categorical, x_continuous)
            
#             if outputs.dim() == 1:
#                 outputs = outputs.unsqueeze(1)
#             probabilities = F.softmax(outputs, dim=1)
#             top_k_prob, top_k_indices = torch.topk(probabilities, k=k, dim=-1)
            
#             all_top_k_indices.extend(top_k_indices.cpu().numpy())
            
#     return all_top_k_indices

# # 모든 데이터에 대해 예측 수행
# all_top_k_indices = predict_all_top_k_products(model, valid_dataloader, k=10)

# # 예측된 모든 product_id의 빈도 집계
# predicted_counts = Counter([idx for sublist in all_top_k_indices for idx in sublist])

# # 가장 자주 예측된 상위 10개 product_id 선정
# top_10_predicted_ids = predicted_counts.most_common(10)
# top_10_predicted_ids = [id_ for id_, count in top_10_predicted_ids]

# print("Most frequently predicted top 10 product IDs:", top_10_predicted_ids)

NameError: name 'predict_all_top_k_products' is not defined

In [48]:
# import torch.nn.functional as F

# # 모델의 예측 함수 정의
# def predict_top_k_products(model, x_categorical, x_continuous, k=10):
#     """
#     모델을 사용하여 상위 k개의 product_id를 예측하는 함수
    
#     Parameters:
#     model (torch.nn.Module): 학습된 모델
#     x_categorical (torch.Tensor): 카테고리형 특성 데이터
#     x_continuous (torch.Tensor): 연속형 특성 데이터
#     k (int): 반환할 상위 product_id의 개수

#     Returns:
#     torch.Tensor: 상위 k개의 product_id에 대한 인덱스
#     """
#     # 모델을 평가 모드로 설정
#     model.eval()
    
#     with torch.no_grad():
#         # 데이터를 모델에 통과시키기
#         outputs = model(x_categorical, x_continuous)
#         # print(outputs)
        
#         if outputs.dim() == 1:
#             outputs = outputs.unsqueeze(1)
#         # Softmax 적용하여 확률 분포 얻기
#         probabilities = F.softmax(outputs, dim=1)
#         # print(probabilities.shape)
        
        
#         # 상위 k개의 확률과 해당 인덱스(=product_id)를 얻기
#         top_k_prob, top_k_indices = torch.topk(probabilities, k=10, dim=-1)

#     return top_k_indices

# # 예측 실행
# # x_categorical과 x_continuous에 대한 데이터 준비가 필요
# # 예: x_categorical, x_continuous = 데이터 로딩 로직
# # device 설정 후 데이터를 device로 이동

# x_categorical = x_categorical.to(device)
# x_continuous = x_continuous.to(device)

# top_k_products = predict_top_k_products(model, x_categorical, x_continuous, k=10)

# print("Top 10 predicted product ids:", top_k_products)

Top 10 predicted product ids: tensor([[11947, 12009, 11640,  6154, 12348,  5791, 10165, 11814,  4993, 12329],
        [10250,   822,   467,  4022,  2851,  1405,  3213,  3227,  1231,   816],
        [ 3224, 11813,  3092,  1009, 10556,  9238,  7993,  2082,  6178,  9891]],
       device='cuda:0')


In [ ]:
# # 상위 k개의 product_id 인덱스를 실제 product_id로 변환
# top_k_product_ids = [[idx2id[idx.item()] for idx in batch] for batch in top_k_products]

# print("Top k predicted product ids:", top_k_product_ids)


입력받은 유저의 product_id에 대한 topk

In [41]:
import torch.nn.functional as F

def prepare_user_product_data(user_product_id, df, categorical_columns, continuous_columns):
    # 유저가 선택한 product_id 추출
    user_product_data = filtered_data[filtered_data['product_id'] == user_product_id]
    
    # 카테고리형 및 연속형 특성을 추출
    X_cat = user_product_data[categorical_columns]
    X_cont = user_product_data[continuous_columns]
    
    # continuous variable 스케일링
    scaler = StandardScaler()
    X_cont_scaled = scaler.fit_transform(X_cont)
    X_cont_scaled = pd.DataFrame(X_cont_scaled, columns=continuous_columns)
    
    return X_cat, X_cont_scaled


# 유저의 product_id입력
user_product_id = 379270836
X_cat_user, X_cont_user = prepare_user_product_data(user_product_id, filtered_data, categorical_columns, continuous_columns)

# 특성 데이터를 모델에 입력 가능한 형태로 변환
X_cat_user_tensor = torch.tensor(X_cat_user.values, dtype=torch.long).to(device)
X_cont_user_tensor = torch.tensor(X_cont_user.values, dtype=torch.float).to(device)

# 모델을 평가 모드로 설정합니다.
model.eval()

# 유저의 product_id에 대한 예측을 수행합니다.
with torch.no_grad():
    output = model(X_cat_user_tensor, X_cont_user_tensor)
    probabilities = F.softmax(output, dim=1)
    top_k_prob, top_k_indices = torch.topk(probabilities, k=10, dim=-1)

# 상위 10개 추천 상품의 인덱스를 추출합니다.
top_k_product_idxs = top_k_indices.cpu().numpy()[0]
top_k_product_ids = [idx2id[idx] for idx in top_k_product_idxs]

print("Top-k recommended product IDs for user's selected product:", top_k_product_ids)


Top-k recommended product IDs for user's selected product: [585333654, 7119313092, 8899487287, 5765258746, 5091364698, 4428569497, 402444256, 4504101657, 4537559776, 5133168785]


In [45]:
filtered_data[filtered_data['product_id'] == 5133168785]

,product_id,product_name,brand,category_1,category_2,category_3,rating,num_review,price,personality,product_idx
174051,5133168785,휴럼사이언스 위간편케어 1개월분 감초추출물 밀크씨슬 함유 헬리코박터균 위 간 건강,2897,3,35,3,4.7,760,25800,2.807355,6487
174055,5133168785,휴럼사이언스 위간편케어 1개월분 감초추출물 밀크씨슬 함유 헬리코박터균 위 간 건강,2897,3,5,3,4.7,760,25800,2.807355,6487
174050,5133168785,휴럼사이언스 위간편케어 1개월분 감초추출물 밀크씨슬 함유 헬리코박터균 위 간 건강,2897,3,4,3,4.7,760,25800,2.807355,6487
174049,5133168785,휴럼사이언스 위간편케어 1개월분 감초추출물 밀크씨슬 함유 헬리코박터균 위 간 건강,2897,1,15,19,4.7,760,25800,2.807355,6487
174053,5133168785,휴럼사이언스 위간편케어 1개월분 감초추출물 밀크씨슬 함유 헬리코박터균 위 간 건강,2897,6,23,1,4.7,760,25800,2.807355,6487
174052,5133168785,휴럼사이언스 위간편케어 1개월분 감초추출물 밀크씨슬 함유 헬리코박터균 위 간 건강,2897,8,23,1,4.7,760,25800,2.807355,6487
174054,5133168785,휴럼사이언스 위간편케어 1개월분 감초추출물 밀크씨슬 함유 헬리코박터균 위 간 건강,2897,0,23,1,4.7,760,25800,2.807355,6487


In [ ]:
# def predict_for_new_input(model, user_input_features):
#     """
#     새로운 유저 입력에 대해 상위 k개 product_id를 예측하고 반환하는 함수

#     Parameters:
#     model (torch.nn.Module): 학습된 모델
#     user_input_features (dict): 새로운 유저 입력에서 추출한 특성 정보

#     Returns:
#     list: 상위 k개의 예측된 product_id
#     """
#     # 유저 입력을 모델의 입력 형태로 변환
#     x_categorical, x_continuous = preprocess_user_input(user_input_features)
    
#     # 모델의 예측 실행
#     top_k_indices = predict_top_k_products(model, x_categorical, x_continuous, k=10)
    
#     # 예측된 인덱스를 실제 product_id로 변환
#     top_k_product_ids = [idx2id[idx.item()] for idx in top_k_indices[0]]
    
#     return top_k_product_ids

# # 새로운 유저 입력에 대한 예시 함수 호출
# user_input_features = ''
# top_k_product_ids = predict_for_new_input(model, user_input_features)


한 세트 --------------

In [ ]:
# class CustomDataset(Dataset):
#     def __init__(self, X, y, cat_features, cont_features):
#         self.X_cat = X[cat_features].astype(np.int64).values
#         self.X_cont = X[cont_features].astype(np.float64).values
#         self.y = y.values
    
#     def __len__(self):
#         return len(self.y)
    
#     def __getitem__(self, idx):
#         return (torch.tensor(self.X_cat[idx], dtype=torch.long),
#                 torch.tensor(self.X_cont[idx], dtype=torch.float),
#                 torch.tensor(self.y[idx], dtype=torch.long))

# train_dataset = CustomDataset(X_train, y_train, categorical_features, continuous_features)
# val_dataset = CustomDataset(X_val, y_val, categorical_features, continuous_features)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# # Adjusting the model
# class WideAndDeepModel(nn.Module):
#     def __init__(self, cat_dims, embed_dim, cont_feature_count, mlp_dims, dropout):
#         super().__init__()
#         self.embeddings = nn.ModuleList([nn.Embedding(dim, embed_dim) for dim in cat_dims])
#         self.cont_feature_adapt = nn.Linear(cont_feature_count, embed_dim)  # Adapting continuous features
#         self.mlp = nn.Sequential(
#             nn.Linear(len(cat_dims) * embed_dim + embed_dim, mlp_dims[0]),  # Adjusted input dimension
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             *[nn.Sequential(nn.Linear(mlp_dims[i], mlp_dims[i+1]), nn.ReLU(), nn.Dropout(dropout)) for i in range(len(mlp_dims)-1)],
#             nn.Linear(mlp_dims[-1], 1)
#         )
    
#     def forward(self, x_cat, x_cont):
#         x_cat = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embeddings)]
#         x_cat = torch.cat(x_cat, dim=1)
#         x_cont = self.cont_feature_adapt(x_cont)
#         x = torch.cat((x_cat, x_cont), dim=1)
#         x = self.mlp(x)
#         return x.squeeze(1)

In [328]:
# model = WideAndDeepModel()

# mode.summary()

TypeError: __init__() missing 5 required positional arguments: 'cat_dims', 'embed_dim', 'cont_feature_count', 'mlp_dims', and 'dropout'

In [267]:
# class GiftDataset(Dataset):
#     def __init__(self, features, labels=None):
#         self.features = features
#         self.labels = labels

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, idx):
#         features = torch.tensor(self.features.iloc[idx].values, dtype=torch.float)  # 실수형 데이터를 처리하기 위해 dtype을 torch.float으로 변경
#         if self.labels is not None:
#             labels = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
#             return features, labels
#         return features

In [ ]:
# class CustomDataset(Dataset):
#     def __init__(self, X, y, cat_features, cont_features):
#         self.X_cat = X[cat_features].astype(np.int64).values
#         self.X_cont = X[cont_features].astype(np.float32).values
#         self.y = y.values

#     def __len__(self):
#         return len(self.y)
    
#     def __getitem__(self, idx):
#         return (torch.tensor(self.X_cat[idx], dtype=torch.long),
#                 torch.tensor(self.X_cont[idx], dtype=torch.float),
#                 torch.tensor(self.y[idx], dtype=torch.long))

In [ ]:
# # feature_fields = ['rating', 'num_review', 'personality'] + category_cols
# train_dataset = GiftDataset(X_train, y_train)
# val_dataset = GiftDataset(X_val, y_val)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


# for batch in train_loader:
#     features, labels = batch
#     print(features)
#     print(labels)
#     print("Features shape:", features.shape)
#     print("Features dtype:", features.dtype)
#     print("Labels shape:", labels.shape)
#     print("Labels dtype:", labels.dtype)
#     break  # 첫 배치만 확인하고 반복문 탈출

In [273]:
# import numpy as np 
# import torch 
# import torch.nn as nn 

In [329]:
# # FM모델 등에서 활용되는 선형 결합 부분을 정의합니다.
# class FeaturesLinear(nn.Module):
#     def __init__(self, field_dims: np.ndarray, output_dim: int=1):
#         super().__init__()
#         self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
#         self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
#         self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)


#     def forward(self, x: torch.Tensor):
#         x = x + x.new_tensor(self.offsets).unsqueeze(0)
#         return torch.sum(self.fc(x), dim=1) + self.bias


# # factorization을 통해 얻은 feature를 embedding 합니다.
# class FeaturesEmbedding(nn.Module):
#     def __init__(self, field_dims: np.ndarray, embed_dim: int):
#         super().__init__()
#         self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
#         self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)
#         torch.nn.init.xavier_uniform_(self.embedding.weight.data)


#     def forward(self, x: torch.Tensor):
#         x = x + x.new_tensor(self.offsets).unsqueeze(0)
#         return self.embedding(x)

In [330]:
# # NCF 모델은 MLP와 GMF를 합하여 최종 결과를 도출합니다.
# # MLP을 구현합니다.
# class MultiLayerPerceptron(nn.Module):
#     def __init__(self, input_dim, embed_dims, dropout, output_layer=True):
#         super().__init__()
#         layers = list()
#         for embed_dim in embed_dims:
#             layers.append(torch.nn.Linear(input_dim, embed_dim))
#             layers.append(torch.nn.BatchNorm1d(embed_dim))
#             layers.append(torch.nn.ReLU())
#             layers.append(torch.nn.Dropout(p=dropout))
#             input_dim = embed_dim
#         if output_layer:
#             layers.append(torch.nn.Linear(input_dim, 1))
#         self.mlp = torch.nn.Sequential(*layers)


#     def forward(self, x):
#         return self.mlp(x)

In [331]:
# class WideAndDeepModel(nn.Module):
#     def __init__(self, field_dims, embed_dim, mlp_dims, dropout):
#         super().__init__()
#         self.linear = FeaturesLinear(field_dims=field_dims, output_dim=1)
#         self.embedding = nn.ModuleList([nn.Embedding(num_embeddings=dim, embedding_dim=embed_dim) for dim in field_dims])
#         self.mlp = nn.Sequential(
#             nn.Linear(sum(field_dims) * embed_dim, mlp_dims[0]),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             *[nn.Sequential(nn.Linear(mlp_dims[i], mlp_dims[i+1]), nn.ReLU(), nn.Dropout(dropout)) for i in range(len(mlp_dims)-1)],
#             nn.Linear(mlp_dims[-1], 1)
#         )

#     def forward(self, x):
#         # 임베딩된 특성을 연결합니다.
#         embedded_features = [embedding(x[:, i].unsqueeze(-1)) for i, embedding in enumerate(self.embedding)]
#         embedded = torch.cat(embedded_features, dim=2).view(x.size(0), -1)
#         # 선형 부분의 출력을 계산합니다.
#         x_linear = self.linear(x)
#         # MLP 부분의 출력을 계산합니다.
#         x_mlp = self.mlp(embedded)
#         # 최종 출력을 계산합니다.
#         output = x_linear + x_mlp
#         return output.squeeze(1)

In [340]:
# # 카테고리, 연속형 동시에 고려하여 deep, wide를 분리한 후 concat하는 방식으로 진행
# # https://github.com/eclarson/MachineLearningNotebooks/blob/master/10.%20Keras%20Wide%20and%20Deep.ipynb
# class WideAndDeepModel(nn.Module):
#     def __init__(self, categorical_field_dims, continuous_field_size, embed_dim, mlp_dims, dropout):
#         super().__init__()
#         # 카테고리 데이터 임베딩
#         self.embeddings = nn.ModuleList([nn.Embedding(num_embeddings, embed_dim) for num_embeddings in categorical_field_dims])
#         self.continuous_field_size = continuous_field_size

#         # Wide 부분
#         self.linear = nn.Linear(continuous_field_size, 1)

#         # Deep 부분
#         total_embed_dim = len(categorical_field_dims) * embed_dim
#         self.deep_input_size = total_embed_dim + continuous_field_size
#         self.mlp = self._make_mlp(self.deep_input_size, mlp_dims, dropout)

#     def _make_mlp(self, input_dim, mlp_dims, dropout):
#         layers = []
#         for output_dim in mlp_dims:
#             layers.append(nn.Linear(input_dim, output_dim))
#             layers.append(nn.ReLU())
#             layers.append(nn.Dropout(dropout))
#             input_dim = output_dim
#         layers.append(nn.Linear(mlp_dims[-1], 1))
#         return nn.Sequential(*layers)

#     def forward(self, x_categorical, x_continuous):
#         x_embed = [embedding(x_categorical[:, i]) for i, embedding in enumerate(self.embeddings)]
#         x_embed = torch.cat(x_embed, dim=1)

#         x_deep = torch.cat([x_embed, x_continuous], dim=1)
#         x_wide = x_continuous

#         wide_output = self.linear(x_wide)
#         deep_output = self.mlp(x_deep)

#         output = wide_output + deep_output
#         return output.squeeze(1)


In [280]:
# # Wide: memorization을 담당하는 generalized linear model
# # Deep: generalization을 담당하는 feed-forward neural network
# # wide and deep model은 위의 wide 와 deep 을 결합하는 모델입니다.
# # 데이터를 embedding 하여 MLP 으로 학습시킨 Deep 모델과 parameter에 bias를 더한 linear 모델을 합하여 최종결과를 도출합니다.
# class WideAndDeepModel(nn.Module):
#     def __init__(self, field_dims, embed_dim, mlp_dims, dropout):
#         super().__init__()
#         self.linear = FeaturesLinear(sum(field_dims), 1)  # 'rating'과 'num_review'만 사용
#         # self.embedding = FeaturesEmbedding(field_dims, embed_dim)
#         self.embedding = nn.ModuleList([nn.Embedding(field_dim, embed_dim) for field_dim in field_dims])
#         # self.embed_output_dim = len(field_dims)* embed_dim
#         # self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout, output_layer=False)
#         self.mlp = nn.Sequential(
#             nn.Linear(len(field_dims) * embed_dim, mlp_dims[0]),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             *[nn.Sequential(nn.Linear(mlp_dims[i], mlp_dims[i+1]), nn.ReLU(), nn.Dropout(dropout)) for i in range(len(mlp_dims)-1)],
#             nn.Linear(mlp_dims[-1], 1)
#         )
#         # self.final_linear = nn.Linear(mlp_dims[-1], 1)

#     def forward(self, x):
#         embedded = torch.cat([embedding(x[:, i]) for i, embedding in enumerate(self.embedding)], dim=1)
#         x_linear = self.linear(x)
#         x_mlp = self.mlp(embedded)
#         # Linear와 MLP 출력의 합
#         output = x_linear + x_mlp
#         return output.squeeze(1)
    
    
#         # print(f'Input Shape: {x.shape}')
#         # linear_part = self.linear(x)
#         # print(f'Linear Part Output Shape: {linear_part.shape}')
#         # embedding_part = self.embedding(x)
#         # print(f'Embedding Part Output Shape: {embedding_part.shape}')
#         # deep_input = torch.cat([embedding_part, x], 1)
#         # deep_output = self.mlp(deep_input)
#         # final_output = self.final_linear(deep_output + linear_part)
#         # return final_output.squeeze(1)

        
#         # combined_input = torch.cat([linear_part, embedding_part], 1)
#         # print(f'Combined Input Shape: {combined_input.shape}')
#         # x = self.mlp(combined_input)
#         # print(f'MLP Output Shape: {x.shape}')
#         # x = self.final_linear(x)
#         # print(f'Final Output Shape: {x.shape}')
#         # return x.squeeze(1)

In [ ]:
# import torch.optim as optim
# from torch.nn import MSELoss
# # 카테고리 데이터에 대해서만 받도록 설정
# field_dims = [df[col].nunique() for col in feature_cols if col in category_cols]

# field_dims=[len(df[col].unique()) for col in feature_cols]

# model = WideAndDeepModel(field_dims, embed_dim=16, mlp_dims=[64, 32], dropout=0.1).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# criterion = MSELoss()